- Hugging Face MRC tutorial notebook을 번역하였습니다. 

원문: https://github.com/huggingface/notebooks/blob/main/examples/question_answering.ipynb

In [1]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.9 MB/s 
     |████████████████████████████████| 4.4 MB 44.6 MB/s 
     |████████████████████████████████| 140 kB 42.8 MB/s 
     |████████████████████████████████| 101 kB 9.4 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 212 kB 47.8 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
     |████████████████████████████████| 127 kB 44.6 MB/s 
     |████████████████████████████████| 6.6 MB 7.3 MB/s 
     |████████████████████████████████| 94 kB 2.0 MB/s 
     |████████████████████████████████| 144 kB 42.2 MB/s 
     |████████████████████████████████| 271 kB 42.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing i

In [2]:
import transformers
from datasets import load_dataset, load_metric
print(f'version of transformers: {transformers.__version__}')

squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

datasets = load_dataset("squad_v2" if squad_v2 else "squad")

version of transformers: 4.20.1


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

SQuAD dataset을 살펴보는 코드입니다.

In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,56cf609aaab44d1400b89187,Frédéric_Chopin,"Fryderyk may have had some piano instruction from his mother, but his first professional music tutor, from 1816 to 1821, was the Czech pianist Wojciech Żywny. His elder sister Ludwika also took lessons from Żywny, and occasionally played duets with her brother. It quickly became apparent that he was a child prodigy. By the age of seven Fryderyk had begun giving public concerts, and in 1817 he composed two polonaises, in G minor and B-flat major. His next work, a polonaise in A-flat major of 1821, dedicated to Żywny, is his earliest surviving musical manuscript.",At what age did Chopin start playing publicly?,"{'text': ['7'], 'answer_start': [391]}"
1,572749b4708984140094dbc3,Affirmative_action_in_the_United_States,"Richard Sander claims that by artificially elevating minority students into schools they otherwise would not be capable of attending, this discourages them and tends to engender failure and high dropout rates for these students. For example, about half of black college students rank in the bottom 20 percent of their classes, black law school graduates are four times as likely to fail bar exams as are whites, and interracial friendships are more likely to form among students with relatively similar levels of academic preparation; thus, blacks and Hispanics are more socially integrated on campuses where they are less academically mismatched. He claims that the supposed ""beneficiaries"" of affirmative action – minorities – do not actually benefit and rather are harmed by the policy. Sander's claims have been disputed, and his empirical analyses have been subject to substantial criticism. A group including some of the country's lead statistical methodologists told the Supreme Court that Sander's analyses were sufficiently flawed that the Court would be wise to ignore them entirely. At the same time many scholars have found that minorities gain substantially from affirmative action.",Who claims that artificially raising minority students into schools has a negative effect on them?,"{'text': ['Richard Sander'], 'answer_start': [0]}"
2,56fa37baf34c681400b0c018,Wood,"Structural material that resembles ordinary, ""dicot"" or conifer wood in its gross handling characteristics is produced by a number of monocot plants, and these also are colloquially called wood. Of these, bamboo, botanically a member of the grass family, has considerable economic importance, larger culms being widely used as a building and construction material in their own right and, these days, in the manufacture of engineered flooring, panels and veneer. Another major plant group that produce material that often is called wood are the palms. Of much less importance are plants such as Pandanus, Dracaena and Cordyline. With all this material, the structure and composition of the structural material is quite different from ordinary wood.","While we call bamboo ""wood,"" what botanical family does it belong to?","{'text': ['grass'], 'answer_start': [241]}"
3,56d1162317492d1400aab8ee,The_Legend_of_Zelda:_Twilight_Princess,"The story focuses on series protagonist Link, who tries to prevent Hyrule from being engulfed by a corrupted parallel dimension known as the Twilight Realm. To do so, he takes the form of both a Hylian and a wolf, and is assisted by a mysterious creature named Midna. The game takes place hundreds of years after Ocarina of Time and Majora's Mask, in an alternate timeline from The Wind Waker.",What is the name of the mysterious creature that assists Link?,"{'text': ['Midna'], 'answer_start': [261]}"
4,572903196aef051400154994,Paris,"Paris is a major rail, highway, and air transport hub. The Syndicat des transports d'Île-de-France (STIF), formerly Syndicat des transports parisiens (STP), oversees the transit network in the region. The syndicate coordinates public transport and contracts it out to the RATP (operating 347 bus lines, the Métro, eight tramway lines, and 

#Preprocessing

- Tokenizer는 사용하고자 하는 LM과 일치해야합니다.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Fase tokenizer는 Rust backend 기반으로 실행되므로, 더 빠르게 tokenization을 수행합니다.

In [7]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [8]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102, 2026, 2171, 2003, 25353, 22144, 2378, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

매우 긴 지문과 어떻게 타협할 지가 중요한 요소입니다. 보통 다른 task에서는 model의 maximum length만큼 잘라서 사용합니다. 그러나 QA task에서는 지문에 대한 답을 잃을 수 있습니다. 

입력 문장을 model의 maximum length보다 짧게 나눠서 input data로 사용할 것입니다. 문장을 자를 때, 겹치는 부분을 고려해야 합니다. 얼마나 겹치게 할지는 `doc_stride`라는 hyper-parameter로 결정합니다.

In [9]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

stride가 작동하는지 확인하기 위해, `max_length`보다 긴 sample을 찾아보겠습니다.

In [10]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        print(i)
        break
example = datasets["train"][i]

249


## `tokenizer`의 `truncation` parameter 

tokenizer에 두 문자열 question과 context가 입력되고 있습니다. `'only_second'`는 두 번째 입력 문자열에 대해서만 truncation을 진행한다는 의미의 parameter입니다.

참고: https://huggingface.co/docs/transformers/pad_truncation

In [11]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second", # 설명 참고!
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride # doc stride 적용!!!
)

In [12]:
[len(x) for x in tokenized_example["input_ids"]] # 두 문자열로 나뉘어졌습니다.

[384, 157]

- input: `question`, `context`

- output: `question + truncated_context_0`, `question + truncated_context_1`

문자열의 첫 문장은 question, 그 이후로 truncated context가 따라옵니다.

In [13]:
tokenizer.decode(tokenized_example['input_ids'][0])

"[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at not

In [14]:
tokenizer.decode(tokenized_example['input_ids'][1])

"[CLS] how many wins does the notre dame men's basketball team have? [SEP] championship. the 2010 – 11 team concluded its regular season ranked number seven in the country, with a record of 25 – 5, brey's fifth straight 20 - win season, and a second - place finish in the big east. during the 2014 - 15 season, the team went 32 - 6 and won the acc conference tournament, later advancing to the elite 8, where the fighting irish lost on a missed buzzer - beater against then undefeated kentucky. led by nba draft picks jerian grant and pat connaughton, the fighting irish beat the eventual national champion duke blue devils twice during the season. the 32 wins were the most by the fighting irish team since 1908 - 09. [SEP]"

### `offset_mapping`

`offset_mapping`은 token이 input context의 몇 번째 문자에 해당하는지 (start, end) 쌍으로 알려줍니다. 

`(0, 0)`: `[CLS]`, `[SEP]` 등의 문장 구조를 나타내기 위한 token은 input context에 존재하지 않으므로 (0, 0)으로 표현됩니다.

In [15]:
print(tokenized_example["offset_mapping"][0][:10])
print(tokenized_example["offset_mapping"][0][-10:])

[(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), (23, 28), (29, 33), (34, 37), (37, 38)]
[(1628, 1633), (1634, 1640), (1641, 1644), (1645, 1651), (1651, 1652), (1653, 1656), (1657, 1659), (1660, 1664), (1665, 1669), (0, 0)]


첫 번째 token을 decode한 결과와 첫 번째 token에 대한 offset idx를 참조한 결과를 비교해보겠습니다.

In [16]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how How


In [17]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [18]:
example['answers']

{'answer_start': [30], 'text': ['over 1,600']}

In [19]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# start token을 탐색할 초기 위치를 정합니다.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token을 탐색할 초기 위치를 정합니다.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

print(f'start:end = {token_start_index}:{token_end_index}')

# 우선 정답이 context 내에 존재하는지 확인합니다.
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # token_start_index와 token_end_index이 정답의 시작과 끝을 가르키도록 움직입니다.
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

start:end = 16:382
23 26


정답의 위치를 token의 시작과 끝 위치로 표현할 수 있게 되었습니다.

In [20]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

over 1, 600
over 1,600


In [21]:
tokenizer.decode(tokenized_example["input_ids"][0])

"[CLS] how many wins does the notre dame men's basketball team have? [SEP] the men's basketball team has over 1, 600 wins, one of only 12 schools who have reached that mark, and have appeared in 28 ncaa tournaments. former player austin carr holds the record for most points scored in a single game of the tournament with 61. although the team has never won the ncaa tournament, they were named by the helms athletic foundation as national champions twice. the team has orchestrated a number of upsets of number one ranked teams, the most notable of which was ending ucla's record 88 - game winning streak in 1974. the team has beaten an additional eight number - one teams, and those nine wins rank second, to ucla's 10, all - time in wins against the top team. the team plays in newly renovated purcell pavilion ( within the edmund p. joyce center ), which reopened for the beginning of the 2009 – 2010 season. the team is coached by mike brey, who, as of the 2014 – 15 season, his fifteenth at not

In [22]:
print(tokenizer.convert_ids_to_tokens(tokenized_example["input_ids"][0])[start_position])
print(tokenizer.convert_ids_to_tokens(tokenized_example["input_ids"][0])[end_position])

over
600


In [23]:
offsets[start_position]

(30, 34)

start, end 위치를 offset을 통해 context 상의 character index로 참조할 수 있습니다.

In [24]:
example['context'][offsets[start_position][0]:offsets[end_position][1]]

'over 1,600'

###Padding

In [25]:
# 일부 model은 context 후에 question이 따르는 순서를 요구할 수도 있습니다. 
# 이에 대한 예외를 처리하는 변수입니다.
pad_on_right = tokenizer.padding_side == "right"

아래 코드는 앞서 설명한 기능들을 하나의 method에 통합하여 구현합니다.

만약 answer가 context에 없을 경우(truncation이 수행된 경우 발생할 수 있습니다.) answer의 start, end index를 `[CLS]`(0, 0)으로 설정합니다. `allow_impossible_answers` parameter를 `False`로 설정하여 위의 경우를 무시할 수도 있습니다.

In [26]:
def prepare_train_features(examples):
    # 일부 questions에는 문자열 좌측에 공백이 많이 존재할수도 있습니다. 
    # 이는 도움이 되지 않으므로 제거합니다.
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # 입력 문장의 길이가 model의 허용량을 초과하더라도, 문장을 쪼개서 데이터를 구성합니다.
    # 원문에서는 '... one example possible giving several features'로 표현하였습니다.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # 만약 하나의 example가 너무 길어서 여러 features로 나뉜다면, 
    # 한 feature가 원래 어떤 example에 연관되었는지 기억할 필요가 있다.
    # sample_mapping이 이를 가능하게 한다.

    # 원문
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # 답이 존재하지 않는 경우 answer에 대한 start, end가 [CLS]를 가르키도록 설정할 것이다.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # sequence_ids를 통해 question와 context를 구분할 수 있다.
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # 정답이 없는 질문일 경우
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # 정답이 지문 내에 존재한다면
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # start, end를 찾는 이전 코드와 같다.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # 정답이 해당 지문에 존재하는지 확인한다.
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                # print(f'out of span, current i: {i}, answer i: {sample_idx}')
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [27]:
features = prepare_train_features(datasets['train'][240:250])

위 method를 dataset 전체에 적용하기 위해 `map`을 적용한다.

In [28]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Parameter 'function'=<function prepare_train_features at 0x7fd776b6fb90> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

# Fine-tuning the model

In [29]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model = model.to(device)

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [30]:
# Training까지 수행해보고 싶다면, huggingface에 가입한 후 token을 발급받아야 합니다.
# 아래 코드를 실행하고 token을 붙여넣습니다.

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [31]:
from transformers import default_data_collator

data_collator = default_data_collator

In [33]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/Koowater/distilbert-base-uncased-finetuned-squad into local empty directory.


In [34]:
# 대략 1시간 30분 정도 소요됩니다.

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 88524
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 16599


Epoch,Training Loss,Validation Loss


Saving model checkpoint to distilbert-base-uncased-finetuned-squad/checkpoint-500
Configuration saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/config.json
Model weights saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-squad/checkpoint-500/special_tokens_map.json
tokenizer config file saved in distilbert-base-uncased-finetuned-squad/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-squad/special_tokens_map.json


KeyboardInterrupt: ignored

In [36]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])